# Extract relationshipts between concepts
Currently not using this

In [2]:
import ollama.client as client
client.list()

In [3]:
mistral_instruct = 'mistral:instruct'
mistral_openorca = 'mistral-openorca:latest'
mcard = client.show(mistral_openorca)
mcard['template']

In [6]:
system_prompt = """
Your task is to find find a relationships between the entities given as a python list.\n
You can use the provided context (delimited between ```) and no other prior knowledge. \n
Follow this chain of thought
Thought 1: What are the two entities as per the context. 
Thought 2: What is the relationship between the given entities as per the given context?. 
Thought 3: Is this a direct relationship as per the context.
Format your response as JSON with the following structure:\n
{ 
    "Entity 1": First Entity,
    "Entity 2": Second Entity,
    "Relationship": Explain the Relationship,
    "Related": 'Yes' only if a direct relation can be infered, otherwise 'No'
}
"""

prompt = """
Entites: ['Cryosurgery', 'non-narcotic pain'].

Context:  ``` 
The two primary types of cancer surgery are open surgery and minimally invasive surgery.

In open surgery, the surgical oncologist makes a large incision, usually to remove all or part of a tumor and some of the surrounding healthy tissue (margins).

Minimally invasive surgical techniques may involve the techniques listed below.

Laparoscopy: A surgical oncologist makes a few small incisions and inserts a laparoscope —a thin tube with a tiny camera attached to it—into one of the incisions to capture an interior image, while inserting surgical tools into the other incisions to excise malignancies and surrounding tissue.

Laser surgery: The surgeon uses a narrow beam of high-intensity light to remove a tumor.

Cryosurgery: The surgeon uses liquid nitrogen to freeze and kill cancer cells.

Robotic surgery: This surgery is similar to a laparoscopic surgery. However, instead of manipulating surgical tools by hand, the surgeon uses a computer console to operate the robotic tools.

Other procedures, such as endoscopies, embolization, Mohs micrographic surgery and pleuroscopies, may be performed by non-surgeons, including dermatologists, radiation oncologists and interventional pulmonologists, depending on the procedure.

Non-surgical treatments may take place before surgery (neoadjuvant therapy) or after surgery (adjuvant therapy) to help prevent cancer growth, metastasis or recurrence. The treatments may include chemotherapy, radiation therapy or hormone therapy.

The Advanced Surgical Recovery Program (ASURE) at City of Hope is designed to help patients recover from surgery more quickly and with fewer complications. ASURE is intended to improve surgical outcomes and enhance the patient experience before, during and after surgery, while also reducing the length of a patient’s hospital stay.

The program is delivered by City of Hope multidisciplinary teams, which include surgeons, anesthesiologists, nutritionists, behavioral health therapists, nurses and other clinicians. They work collaboratively to implement evidenced-informed protocols, all under one roof. ASURE combines post-surgical rehabilitation with pre-habilitation, by preparing patients before surgery.

Typically, ASURE protocols involve nutrition, pain management with non-narcotic pain relievers and physical activity within 24 hours after surgery. The use of enhanced recovery methods like those used in ASURE has been shown to reduce fatigue, post-surgery complications and the amount of narcotics used in post-surgical care, while helping patients resume their everyday activities more quickly.
```
"""

combined_prompt = system_prompt + "\n" + prompt
response, context = client.generate(
    model_name=mistral_openorca, prompt=prompt, system=system_prompt
)

In [74]:
import pandas as pd
import json
# from helpers.prompts import graphPrompt

def graphPrompt(input: str, metadata={}, model="mistral-openorca:latest"):
    SYS_PROMPT = (
        "You are a network graph maker. Your task is to extract a knowledge graph of ontologies from the given text. "
        "The graph is made in such a way that, nodes represent the key concepts discussed in the text. "
        "The edges of the graph represent the relation between two concepts.\n"
        "Thought 1: Think about the key concepts or entities mentioned in the text."
        "           The concepts may include object, entity, location, organisation, person, condition, acronym, documents, services, concept, etc.\n"
        "Thought 2: Find out the pairs of concepts that are strongly related to each other.\n "
        "Thought 3: Find out the relation between each pairs. \n"
        "Format your output as a list of json. Each element of the list contains two nodes "
        "and a relation between them, like the follwing: \n"
        "[\n"
        "   {\n"
        "       \"node_1\": \"first concept or entity\",\n"
        "       \"node_2': \"second concept or entity\",\n"
        "       \"edge': \"relationship between the two concepts, node_1 and node_2\"\n"
        "   }, {...}\n"
        "]"
    )
    response, _ = client.generate(model_name=model, system=SYS_PROMPT, prompt=input)
    try:
        result = json.loads(response)
        result = [dict(item, **metadata) for item in result]
    except BaseException as error:
        print("\n\nERROR ### Here is the buggy response: ", response, "\n\n")
        print(error)
        result = None
    return result

input = """
Health data interoperability means the ability of different health information technology systems to communicate and exchange data with each other seamlessly, securely, and efficiently with the Patient’s consent. Interoperability enables health data to be shared among healthcare providers, hospitals, and other healthcare organisations, regardless of the technology platform they are using.
Simply speaking, that means you should not be needed to lug your entire medical history in hard copy to every doctor, every time you sneeze. 
We have all experienced going to a doctor carrying that big fat folder, filed with withering pages of our medical history. And when the doctor asks questions about the missing information, so hard we try to recollect the name of the drug prescribed by the last doctor, or our allergies (Thou shan’t forget those!). And still coming out of the cabin only to remember a crucial detail we forgot to mention… Dam! 
Well… that was an oversimplified — and slightly exaggerated for dramatic effect — reconstruction of my medical experiences visiting a doctor. But it drives the point home. Remember the mobile charger hell… it is like that, only the stakes are higher. 
Every healthcare service provider these days uses a technology platform to manage its services. This includes a Hospital management system (HMS), Electronic Health Record (EHR), Patient Record (PHR) system etc. In practical terms, health data interoperability means that healthcare information can be accessed by authorised providers across different systems, regardless of where the information was originally generated or stored. This includes data such as medical histories, lab results, imaging studies, Insurance coverage, and other clinical information.
Apart from healthcare providers like a hospital or doctor’s clinics, The purview of Health Data Interoperability includes Diagnostic labs, Insurance providers, Pharmacies, Wellness Service providers (Gyms, Yoga centres), Mobile Health and Wellness (HnW) apps, Wearable devices (iPhone, Fitbit, Glucometer), Government Healthcare Schemes like CGHS etc, and even public health research. 
The lack of interoperability has been a major barrier to the adoption of health information technology and the sharing of patient data in the past. 
But before we dig deeper into that, we must Quench the Question that is Quintessential to the Quest for a change from the Quiescence. (Wow.. what a Quip.. right?)

"""

res = graphPrompt(input)

In [75]:
res

In [76]:
df = pd.DataFrame(res)
df